# Final State Radiation

In this notebook, we use `hazma` to compute final state radiation (FSR) for dark matter fermions $\chi$ annihilating to electron, muons and pions. The easiest way to get to the FSR functions is to first declare the paramaters of the model, then compute a mediator object. For example:
```
sm_params = {"mx" : 250., "ms" : 600., "gsxx" :  1., "gsff" :  1., "gsGG" :  1., "gsFF" :  1.}
SM = ScalarMediator(**params)
```
Then, we need to specify our gamma-ray energies and our center of mass energy, all in MeV:
```
cme = 1000.
egams = numpy.logspace(-3., 3. , num=150)
```
We can then compute the spectrum values at those gamma-ray energies by calling, for example:
```
sm_fsr_e = dnde_xx_to_s_to_ffg(egams, cme, me, SM)
```

## Imports

The mediator classes are stored in `hazma.$mediator` where `$mediator` = `scalar`, `vector`. The FSR functions are stored in `hazma.$mediator.$mediator_mediator_fsr`.

In [ ]:
from hazma.scalar_mediator import ScalarMediator
from hazma.vector_mediator import VectorMediator

from hazma.scalar_mediator.scalar_mediator_fsr import dnde_xx_to_s_to_ffg, dnde_xx_to_s_to_pipig
from hazma.vector_mediator.vector_mediator_fsr import dnde_xx_to_v_to_ffg, dnde_xx_to_v_to_pipig

from hazma.parameters import electron_mass as me
from hazma.parameters import muon_mass as mmu
from hazma.parameters import charged_pion_mass as mpi

from hazma.field_theory_helper_functions.three_body_phase_space import t_lim1, t_lim2

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def mx_to_cme(mx, v):
    return 2. * mx * (1.0 + 0.5 * v**2 )

## Scalar Mediator FSR

First, we specify the scalar mediator couplings at the 1 GeV scale and make a scalar mediator object.

In [ ]:
sm_params = {"mx" : 250., "ms" : 1000., "gsxx" :  1., "gsff" :  1., "gsGG" :  1., "gsFF" :  1.}

SM = ScalarMediator(**sm_params)

Next, we declare the gamma ray energies and the center of mass energy and compute the FSR spectra.

In [ ]:
cme = mx_to_cme(sm_params["mx"], 1e0)
egams = np.logspace(-2., np.log10(cme), num=150)

sm_fsr_e = dnde_xx_to_s_to_ffg(egams, cme, me, SM)
sm_fsr_mu = dnde_xx_to_s_to_ffg(egams, cme, mmu, SM)
sm_fsr_pi = dnde_xx_to_s_to_pipig(egams, cme, SM)

Now we can plot!

In [ ]:
plt.figure(dpi=100)
plt.loglog(egams, sm_fsr_e, label=r"$X = e^{+}e^{-}$")
plt.loglog(egams, sm_fsr_mu, label=r"$X = \mu^{+}\mu^{-}$")
plt.loglog(egams, sm_fsr_pi, label=r"$X = \pi^{+}\pi^{-}$")
plt.title(r"$\bar{\chi}\chi\to S^{*}\to X$")
plt.xlabel(r"$E_{\gamma}$ (MeV)")
plt.ylabel(r"$\frac{dN}{dE_{\gamma}}$ (MeV$^{-1}$)")
plt.legend()

## Vector Mediator FSR

In [ ]:
vm_params = {"mx" : 250., "mv" : 600., "gvxx" : 1.,"gvuu" : 1., "gvdd" : -1., 
             "gvss" : 1., "gvee" : 1., "gvmumu" : 1.}

VM = VectorMediator(**vm_params)

In [ ]:
cme = mx_to_cme(vm_params["mx"], 1e-3)
egams = np.logspace(-2., np.log10(cme), num=150)

vm_fsr_e = dnde_xx_to_v_to_ffg(egams, cme, "e", VM)
vm_fsr_mu = dnde_xx_to_v_to_ffg(egams, cme, "mu", VM)
vm_fsr_pi = dnde_xx_to_v_to_pipig(egams, cme, VM)

In [ ]:
plt.figure(dpi=100)
plt.loglog(egams, vm_fsr_e, label=r"$X = e^{+}e^{-}$")
plt.loglog(egams, vm_fsr_mu, label=r"$X = \mu^{+}\mu^{-}$")
plt.loglog(egams, vm_fsr_pi, label=r"$X = \pi^{+}\pi^{-}$")
plt.title(r"$\bar{\chi}\chi\to V^{*}\to X$")
plt.xlabel(r"$E_{\gamma}$ (MeV)")
plt.ylabel(r"$\frac{dN}{dE_{\gamma}}$ (MeV$^{-1}$)")
plt.legend()

## Compare with Alterelli-Parisi

In [ ]:
sm_params = {"mx" : 100., "ms" : 600., "gsxx" :  1., "gsff" :  1., "gsGG" :  1., "gsFF" :  1.}
vm_params = {"mx" : 100., "mv" : 600., "gvxx" : 1.,"gvuu" : 1., "gvdd" : -1., 
             "gvss" : 1., "gvee" : 1., "gvmumu" : 1.}

SM = ScalarMediator(**sm_params)
VM = VectorMediator(**vm_params)

In [ ]:
def AP(eng_gam, mass, cme):
    x0 = 2. * eng_gam / cme
    eps = 2 * mass / cme
    pre_fac = 1.0 / (137.0 * np.pi) * 2. / cme
    split_func = (1. + (1. - x0)**2) / x0
    log_sup = (-1. + np.log(4. * (1 - x0) / eps**2))
    return pre_fac * split_func * log_sup

apvec = np.vectorize(AP)

### Muon

In [ ]:
vrel = 1.0e-3

# ########################################################

mx = 110.
sm_params["mx"] = mx
vm_params["mx"] = mx
SM = ScalarMediator(**sm_params)
VM = VectorMediator(**vm_params)
cme = mx_to_cme(mx, vrel)

egams = np.logspace(-2., np.log10(cme), num=150)
sm_fsr_mu_215 = dnde_xx_to_s_to_ffg(egams, cme, mmu, SM)
vm_fsr_mu_215 = dnde_xx_to_v_to_ffg(egams, cme, "mu", VM)
ap_fsr_mu_215 = AP(egams, mmu, cme)

# ########################################################

mx = 125.
sm_params["mx"] = mx
vm_params["mx"] = mx
SM = ScalarMediator(**sm_params)
VM = VectorMediator(**vm_params)
cme = mx_to_cme(mx, vrel)

egams = np.logspace(-2., np.log10(cme), num=150)
sm_fsr_mu_250 = dnde_xx_to_s_to_ffg(egams, cme, mmu, SM)
vm_fsr_mu_250 = dnde_xx_to_v_to_ffg(egams, cme, "mu", VM)
ap_fsr_mu_250 = AP(egams, mmu, cme)

# ########################################################

mx = 150.
sm_params["mx"] = mx
vm_params["mx"] = mx
SM = ScalarMediator(**sm_params)
VM = VectorMediator(**vm_params)
cme = mx_to_cme(mx, vrel)

egams = np.logspace(-2., np.log10(cme), num=150)
sm_fsr_mu_300 = dnde_xx_to_s_to_ffg(egams, cme, mmu, SM)
vm_fsr_mu_300 = dnde_xx_to_v_to_ffg(egams, cme, "mu", VM)
ap_fsr_mu_300 = AP(egams, mmu, cme)

# ########################################################

mx = 250.
sm_params["mx"] = mx
vm_params["mx"] = mx
SM = ScalarMediator(**sm_params)
VM = VectorMediator(**vm_params)
cme = mx_to_cme(mx, vrel)

egams = np.logspace(-2., np.log10(cme), num=150)
sm_fsr_mu_500 = dnde_xx_to_s_to_ffg(egams, cme, mmu, SM)
vm_fsr_mu_500 = dnde_xx_to_v_to_ffg(egams, cme, "mu", VM)
ap_fsr_mu_500 = AP(egams, mmu, cme)

In [ ]:
f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex='col', sharey='row', figsize=(7.5, 5.5), squeeze=True)

f.dpi=200

ax1.loglog(egams, egams * sm_fsr_mu_215, label=r"SM")
ax1.loglog(egams, egams * vm_fsr_mu_215, label=r"VM")
ax1.loglog(egams, egams * ap_fsr_mu_215, label=r"AP")

ax2.loglog(egams, egams * sm_fsr_mu_250, label=r"SM")
ax2.loglog(egams, egams * vm_fsr_mu_250, label=r"VM")
ax2.loglog(egams, egams * ap_fsr_mu_250, label=r"AP")

ax3.loglog(egams, egams * sm_fsr_mu_300, label=r"SM")
ax3.loglog(egams, egams * vm_fsr_mu_300, label=r"VM")
ax3.loglog(egams, egams * ap_fsr_mu_300, label=r"AP")

ax4.loglog(egams, egams * sm_fsr_mu_500, label=r"SM")
ax4.loglog(egams, egams * vm_fsr_mu_500, label=r"VM")
ax4.loglog(egams, egams * ap_fsr_mu_500, label=r"AP")


# Plot labels
ax1.text(1.3e-2, 1.2e-4, r"$m_{\chi}$ = 110 MeV", fontsize=14)
ax2.text(1.3e-2, 1.2e-4, r"$m_{\chi}$ = 125 MeV", fontsize=14)
ax3.text(1.3e-2, 1.2e-3, r"$m_{\chi}$ = 150 MeV", fontsize=14)
ax4.text(1.3e-2, 1.2e-3, r"$m_{\chi}$ = 250 MeV", fontsize=14)

# Axes Labels
ax3.set_xlabel(r"$E_{\gamma}$ (MeV)", fontsize=14)
ax4.set_xlabel(r"$E_{\gamma}$ (MeV)", fontsize=14)

ax1.set_ylabel(r"$E_{\gamma}\frac{dN}{dE_{\gamma}}$ ", fontsize=14)
ax3.set_ylabel(r"$E_{\gamma}\frac{dN}{dE_{\gamma}}$ ", fontsize=14)

# Axes ylim
ax3.set_ylim([1e-3,2e-2])

# Axes xlim
ax3.set_xlim(egams[[0, -1]])
ax4.set_xlim(egams[[0, -1]])

# Remove whitespace
plt.subplots_adjust(wspace=0.05, hspace=0.1)

# Add watermark
# plt.text(0.00001, 10000, "Preliminary", fontsize=80,rotation=-45, alpha=0.2, color="r")

ax3.legend()
plt.suptitle(r"$\bar{\chi}\chi\to\mu^{+}\mu^{-}$, $v_{\mathrm{rel}} = 10^{-3}$", y=0.92)

plt.savefig("figures/muon_fsr.pdf")

### Charged Pion

In [ ]:
vrel = 1.0e-3

mx = 150.
sm_params["mx"] = mx
vm_params["mx"] = mx
SM = ScalarMediator(**sm_params)
VM = VectorMediator(**vm_params)
cme = mx_to_cme(mx, vrel)

egams = np.logspace(0., np.log10(cme), num=150)
sm_fsr_pi_300 = dnde_xx_to_s_to_pipig(egams, cme, SM)
vm_fsr_pi_300 = dnde_xx_to_v_to_pipig(egams, cme, VM)
ap_fsr_pi_300 = AP(egams, mpi, cme)
sm_fsr_pi_3002 = dndx_vec(egams, cme)

mx = 250.
sm_params["mx"] = mx
vm_params["mx"] = mx
SM = ScalarMediator(**sm_params)
VM = VectorMediator(**vm_params)
cme = mx_to_cme(mx, vrel)

egams = np.logspace(0., np.log10(cme), num=150)
sm_fsr_pi_500 = dnde_xx_to_s_to_pipig(egams, cme, SM)
vm_fsr_pi_500 = dnde_xx_to_v_to_pipig(egams, cme, VM)
ap_fsr_pi_500 = AP(egams, mpi, cme)
sm_fsr_pi_5002 = dndx_vec(egams, cme)

In [ ]:
f, ((ax1, ax2)) = plt.subplots(1, 2, sharex='col', sharey='row', figsize=(10,4), squeeze=True)

f.dpi=200

ax1.loglog(egams, egams * sm_fsr_pi_300, label=r"SM")
#ax1.loglog(egams, egams * sm_fsr_pi_3002, label=r"SM2")
#ax1.loglog(egams, egams * sm_fsr_pi_3003, label=r"SM3")

ax1.loglog(egams, egams * vm_fsr_pi_300, label=r"VM")
ax1.loglog(egams, egams * ap_fsr_pi_300, label=r"AP")

ax2.loglog(egams, egams * sm_fsr_pi_500, label=r"SM")
#ax2.loglog(egams, egams * sm_fsr_pi_5002, label=r"SM2")
#ax2.loglog(egams, egams * sm_fsr_pi_5003, label=r"SM3")

ax2.loglog(egams, egams * vm_fsr_pi_500, label=r"VM")
ax2.loglog(egams, egams * ap_fsr_pi_500, label=r"AP")


# ylims

ax1.set_ylim([1e-5, 1e-2])
ax2.set_ylim([1e-5, 1e-2])

ax1.set_xlim([1e0, 3e2])
ax2.set_xlim([1e0, 3e2])

# Plot labels
ax1.text(1.5e0, 2e-5, r"$m_{\chi}$ = 150 MeV", fontsize=14)
ax2.text(1.5e0, 2e-5, r"$m_{\chi}$ = 250 MeV", fontsize=14)

# Axes Labels
ax1.set_xlabel(r"$E_{\gamma}$ (MeV)", fontsize=14)
ax2.set_xlabel(r"$E_{\gamma}$ (MeV)", fontsize=14)

ax1.set_ylabel(r"$E_{\gamma}\frac{dN}{dE_{\gamma}}$", fontsize=14)

# Remove whitespace
plt.subplots_adjust(wspace=0.1, hspace=0.1)

ax2.legend(loc=6)

plt.savefig("figures/chrdpi_fsr.pdf")

In [ ]:
sm_fsr_pi_5002[0] / sm_fsr_pi_500[0]

In [ ]:
sm_fsr_pi_3002[0] / sm_fsr_pi_300[0]

In [ ]:
from cmath import sqrt, log, pi
from hazma.parameters import qe, vh, b0

from hazma.scalar_mediator.scalar_mediator_cross_sections import sigma_xx_to_s_to_pipi
from hazma.scalar_mediator.scalar_mediator_widths import width_s_to_pipi

from hazma.field_theory_helper_functions.three_body_phase_space import E1_to_s
from hazma.field_theory_helper_functions.three_body_phase_space import t_integral
from hazma.field_theory_helper_functions.common_functions import \
    cross_section_prefactor
from hazma.field_theory_helper_functions.three_body_phase_space import \
    phase_space_prefactor, t_lim1, t_lim2

In [ ]:
def dndx(egam, Q):
    mupi = mpi / Q
    x = 2 * egam / Q
    
    if x < 0. or 1. - 4. * mupi**2 < x:
        return 0.
    
    dynamic = (2*sqrt(-1 + x)*sqrt(-1 + 4*mupi**2 + x) + 
               (-1 + 2*mupi**2 + x)*
               log((1 - x + sqrt(-1 + x)*
                    sqrt(-1 + 4*mupi**2 + x))**2/\
                   (-1 + x + sqrt(-1 + x)*sqrt(-1 + 4*mupi**2 + x))**2))/x
    coeff = qe**2/(4.*sqrt(1 - 4*mupi**2)*pi**2)
    
    ret_val = 2. * dynamic * coeff / Q
    
    return ret_val.real

dndx_vec = np.vectorize(dndx)